 # Figures for Martijn / RSF / HCO3/Co2

In [14]:
# set working directory to main directory of package
import os

from matplotlib.pyplot import ylabel
try:
    current_file_dir = Path(__file__).parent
except NameError:
    current_file_dir = os.getcwd()

from pathlib import Path
from set_cwd_to_project_root import project_root

import pandas as pd
import numpy as np
import plotting

from chemistry_tools.formulae.formula import Formula
mw_hco3 = Formula.from_string('HCO3').mass
mw_nahco3 = Formula.from_string('NaHCO3').mass
mw_co2 = Formula.from_string('CO2').mass
mw_ca = Formula.from_string('CO2').mass
from phreeqpython.phreeqpython import PhreeqPython
from phreeqc_databases_and_definitions.cdmusic_surface import CDMusicSurface
from phreeqc_databases_and_definitions.common_functions import run_over_ph_range

pd.options.plotting.backend = "plotly"
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
current_file_dir

'D:\\Users\\korevma\\Documents\\Repositories\\coagualation'

 # Solution in tank

In [46]:
pp=PhreeqPython()#project_root / 'phreeqc_databases_and_definitions' / 'phreeqc.dat')
sol_dict = {
  'Temp':      20,
  'pH':        7.3,
  'units':     'mg/kgw',
#   'Amm': '3.3 ',  # is removed?
  'Ca': '127 ',
  'Cl': '32.8 charge',
  '[Fe+2]': '0.03', #'11.9 ', # is removed? still 11 in synthetic water...
  'K':         "2 ",
  'Mg': '10.0 ',
  'Na': '17.9 ',
  'P': "0.98 ",
  'Si': '17.15  as SiO3',
  'S(6)': '0.002  as SO4',
  'Alkalinity': "290.0 as HCO3 ", # how is this determined, as alkalinity via titration
}
sol_tank=pp.add_solution(sol_dict)
solutions=dict(tank=sol_tank)
plotting.createSpeciesPlot(solutions=solutions.values(), labels=solutions.keys(), render_plot=False, )#only_contain_species='F' )


In [47]:
plotting.createSIPlot(solutions=[sol_tank], labels=['tank'], render_plot=False, )#only_contain_species='F' )


 ## Check for charge balance (crude way)
 increase in Cl concentration to get charge balanced solution. Should be small

In [49]:
sol_charge_bal_dict = sol_dict.copy()
sol_charge_bal_dict['Cl'] = f"{sol_dict['Cl']}".replace('charge', '')
sol_charge_bal = pp.add_solution(sol_charge_bal_dict)
100 * (sol_charge_bal.elements['Cl'] - sol_tank.elements['Cl']) /sol_tank.elements['Cl']

-71.1474570855656

In [50]:
[sol_charge_bal.elements['Cl'], sol_tank.elements['Cl']]

[0.0009252468265162194, 0.0032068120624935818]

# Solution properties after lowering pH and equilibrate with Calcite, CO2

First pH is lowered so that SI of calcite become negative for 20% gas mixture, then equilibrate with co2 and calcite

In [52]:
sol_eq_co2 = {'tank': sol_tank}
start_ph = 6.0
# sol_eq_co2.saturate(['Calcite', 'CO2(g)'], [0, -3])
for fracion in [0.10, 0.20, 0.30, 1.00]:
    key = f'{fracion*100}% air'
    sol_eq_co2[key] = sol_tank.copy()
    sol_eq_co2[key].change_ph(start_ph)
    sol_eq_co2[key].equalize(['Calcite', 'CO2(g)', 'O2(g)'],
                       [0, np.log10(fracion*0.039), np.log10(fracion*0.2)],
                       [0, 10, 10])
# plotting.createSIPlot(solutions= list(sol_eq_co2.values()),
#                            labels= list(sol_eq_co2.keys()),
#                            render_plot=False,
#                            min_concentration=-8,
#                            # only_contain_species='F'
#                            )
pd.DataFrame({k: [v.si('Calcite'), v.pH, v.total('HCO3-', units='mmol')*mw_hco3, 
                  v.total('CO2', units='mmol')*mw_co2, v.total('Ca+2', units='mmol')*mw_ca, 
                  v.I]
             for k, v in sol_eq_co2.items()},
             index=['SI', 'pH', 'HCO3 [mg/L]','CO2[mg/L]', 'Ca[mg/L]', 'IS'])

,tank,10.0% air,20.0% air,30.0% air,100.0% air
SI,0.294605,-0.102664,-0.396973,-0.569908,-1.085817
pH,7.300000,7.350399,7.052524,6.877916,6.358340
HCO3 [mg/L],278.978547,97.460910,98.172212,98.506947,99.244227
CO2[mg/L],21.841822,6.789551,13.578764,20.367663,67.881268
Ca[mg/L],133.866241,137.163216,137.315482,137.387603,137.544817
IS,0.011280,0.011453,0.011461,0.011465,0.011474


## Determine how much inorganic carbon we need to get alkalinity of 290 at pH=7.2

do it the dumb and crude way starting at 20% air and add hco3 until we reach hco3 we require

In [53]:
sol = sol_eq_co2['20.0% air'].copy()
nahco3_dose = 2 # mg/L
total_nahco3_dose = 0 # mg/L
while sol.total('HCO3', units='mmol') * mw_hco3 < 290:
    sol.add('NaHCO3', nahco3_dose / mw_nahco3, units='mmol')
    sol.change_ph(7.2)
    total_nahco3_dose +=nahco3_dose
print(f"pH={sol.pH:.2f}, CO2={sol.total('CO2', units='mmol') * mw_co2:.2f} mg/L HCO3={sol.total('HCO3', units='mmol') * mw_hco3:.2f} mg/L")
print(f"total dose of nahco3 is {total_nahco3_dose} mg/L")
print(f"total amount of inorganic carbon is {sol.elements['C(4)']*1000:.2f} mmol/L")

pH=7.20, CO2=27.48 mg/L HCO3=290.18 mg/L
total dose of nahco3 is 288 mg/L
total amount of inorganic carbon is 5.39 mmol/L


## Determine how much NaHCO3 and NaOH to add for each gas mixture and

We require 5.42 mmol/L of inorganic C (see above)

In [54]:

sol_eq_co2_then_ph72 = {}
add_nahco3={}
add_naoh={}
add_hcl={}
add = {}
target_ph = 7.2
required_inorganic_c = 5.41e-3 # mol/L
# sol_eq_co2_then_ph72.saturate(['Calcite', 'CO2(g)'], [0, -3])
for fraction in [0.20, 1.00]:
    key = f'{fraction*100}% air'
    _sol = sol_eq_co2[key].copy()
    sol_eq_co2_then_ph72[key] = _sol
    inorganic_c = _sol.elements['C(4)'] # mol/L
    required_addition_of_inorganic_c = required_inorganic_c - inorganic_c
    _sol.add('NaHCO3', required_addition_of_inorganic_c, 'mol')
    print(f"total inorganic carbon {_sol.elements['C(4)'] * 1000:.2f} mmol/L")
    na_before = _sol.elements['Na']
    cl_before = _sol.elements['Cl']
    _sol.change_ph(target_ph)
    na_after = _sol.elements['Na']
    cl_after = _sol.elements['Cl']
    add_nahco3[key] = required_addition_of_inorganic_c * 1000 * mw_nahco3
    add_naoh[key] = (na_after-na_before) * 1000 # mmol/L
    add_hcl[key] = (cl_after-cl_before) * 1000 # mmol/L
    add['naoh'] = add_naoh
    add['hcl'] = add_hcl
    add['nahco3 [mg/L]'] = add_nahco3
    
    print(f"{key}: change in Na is {(na_after-na_before)*1000:.2f} mmol/L, change in Cl is {(cl_after-cl_before)*1000:.2f} mmol/L)")
    # sol_eq_co2_then_ph72[key].equalize(['Calcite', 'CO2(g)', 'O2(g)'],
    #                    [0, np.log10(fracion*0.039), np.log10(fracion*0.2)],
    #                    [0, 10, 10])
# plotting.createSIPlot(solutions=[sol_tank] + list(sol_eq_co2_then_ph72.values()),
#                            labels=['tank'] + list(sol_eq_co2_then_ph72.keys()),
#                            render_plot=False,
#                            min_concentration=-8,
#                            # only_contain_species='F'
#                            )
pd.DataFrame({k: [v.si('Calcite'), v.pH, v.total('HCO3-', units='mmol')*mw_hco3, 
                  v.total('CO2', units='mmol')*mw_co2, v.total('Ca+2', units='mmol')*mw_ca, 
                  v.I]
             for k, v in {'tank': sol_tank, **sol_eq_co2_then_ph72}.items()},
             index=['SI', 'pH', 'HCO3-','CO2', 'Ca', 'IS'])

total inorganic carbon 5.41 mmol/L
20.0% air: change in Na is 0.00 mmol/L, change in Cl is 0.31 mmol/L)
total inorganic carbon 5.41 mmol/L
100.0% air: change in Na is 0.94 mmol/L, change in Cl is -0.00 mmol/L)


,tank,20.0% air,100.0% air
SI,0.294605,0.175746,0.177816
pH,7.300000,7.200000,7.200000
HCO3-,278.978547,283.365614,283.338851
CO2,21.841822,27.570972,27.597798
Ca,133.866241,134.170267,134.149437
IS,0.011280,0.014735,0.014429


### How much NaOH, HCl, NaHCO3 is added?

In [61]:
pd.DataFrame(add)

,naoh,hcl,nahco3 [mg/L]
20.0% air,1.734723e-15,3.053529e-01,289.403410
100.0% air,9.433651e-01,-4.770490e-14,184.452907


 # Synthetic water
 Repeat what we have done above for synthetic water instead of the real water from Spannenburg

 # Create the original synthetic water at pH=3.5

In [67]:
sol_synth_dict = {
  'Temp':      20,
  'pH':        '7 charge',
  'units':     'mmol/kgw',
  'Amm': '0.18',
  'Ca': '3.03',
  'Cl': '7.4',
  '[Fe+2]': '0.21',
  'K':         '0.05',
  'Mg': '0.42',
  'Mn': '0.009',
  'Na': '0.52 ',
  'P': "0.01 ",
  'Si': '0.23',
  'S(6)': '0.02',
  'N(3)': '0.154'

#   'Alkalinity': "437.0 mg/kgW as HCO3 ",
}
sol_synt = pp.add_solution(sol_synth_dict)
sol_synths = dict(original= sol_synt)
sol_synt_after_hcl = sol_synt.copy()
volume_reactor = 60# liter
hcl_dose = 33 # mmol , originally 50 mmol
na_hco3_dose = 0.1 # mmol / L
sol_synths['after hcl'] = sol_synt.copy().add('HCl', hcl_dose/volume_reactor, 'mmol') # 50 mmol HCl toegevoegd aan reactor met volume van 60 Liter
sol_synths['after hcl and add hco'] = sol_synths['after hcl'].copy().add('NaHCO3', na_hco3_dose, 'mmol')

fracion=1.
sol_synths['after hcl and bubble 100% CO2'] = sol_synths['after hcl'].copy()
sol_synths['after hcl and bubble 100% CO2'].equalize(['Calcite', 'CO2(g)', 'O2(g)'],
                    [0, np.log10(fracion*0.039), np.log10(fracion*0.2)],
                    [0, 10, 10])
fracion=0.2
sol_synths['after hcl and bubble 20% CO2'] = sol_synths['after hcl'].copy()
sol_synths['after hcl and bubble 20% CO2'].equalize(['Calcite', 'CO2(g)', 'O2(g)'],
                    [0, np.log10(fracion*0.039), np.log10(fracion*0.2)],
                    [0, 10, 10])

# sol_synths['after hcl and bubble CO2 and add hco3'] = sol_synths['after hcl and bubble CO2'].copy().add('NaHCO3', na_hco3_dose, 'mmol')
pd.DataFrame({k: [v.si('Calcite'), v.pH, v.total('HCO3-', units='mmol')*mw_hco3, 
                  v.total('CO2', units='mmol')*mw_co2, v.total('Ca+2', units='mmol')*mw_ca, 
                  v.I]
             for k, v in sol_synths.items()},
             index=['SI', 'pH', 'HCO3 [mg/L]','CO2[mg/L]', 'Ca[mg/L]', 'IS'])


,original,after hcl,after hcl and add hco,after hcl and bubble 100% CO2,after hcl and bubble 20% CO2
SI,-999.000000,-999.000000,-1.591310,-5.502073,-6.102657
pH,10.229157,5.366841,7.154186,4.158420,4.207529
HCO3 [mg/L],0.000000,0.000000,5.164727,0.627128,0.140467
CO2[mg/L],0.000000,0.000000,0.564389,67.872071,13.576908
Ca[mg/L],132.561698,133.171919,132.988586,133.166490,133.174468
IS,0.011539,0.011764,0.011849,0.011835,0.011827


 Amount of NaHCO3 added:

In [ ]:
print(f"{na_hco3_dose:.2f} mmol/L (={na_hco3_dose*mw_nahco3:.2f} mg/L) NaHCO3 added to the reactor")